In [0]:
%run ./01-config

In [0]:
from pyspark.sql import functions as F
landing_zone = base_dir_data + "/raw"
checkpoint_base = base_dir_checkpoint + "/checkpoints"






spark.sql(f"USE {catalog}.{db_name}")



def consume_user_registration(once = True, processing_time = "5 seconds"):
    schema = "user_id long, device_id long, mac_address string, registration_timestamp double"

    df_stream = (spark.readStream
                    .format("cloudfiles")
                    .schema(schema)
                    .option("cloudFiles.maxFilesPerTrigger", 1)
                    .option("cloudFiles.format", "csv")
                    .option("header", True)
                    .load(landing_zone + "/registered_users_bz")
                    .withColumn("load_time", F.current_timestamp())
                    .withColumn("source_file", F.input_file_name())

                
                )

    #Use append mode because bronze layer is expected to insert only from source

    stream_writer = df_stream.writeStream \
        .format("delta") \
        .option("checkpointLocation", checkpoint_base + "/registered_users_bz") \
        .outputMode("append")\
        .queryName("dev.sbit_db.registered_users_bz")


    if once == True:
        return stream_writer.trigger(availableNow=True).toTable(f"{catalog}.{db_name}.registered_users_bz")
    else:
        return stream_writer.trigger(processingTime=processing_time).toTable(f"{catalog}.{db_name}.registered_users_bz")



def consume_gym_logins(once = True, processing_time = "5 seconds"):
    schema = "mac_address string, gym bigint, login double, logout double"

    df_stream = (spark.readStream 
                    .format("cloudFiles") 
                    .schema(schema) 
                    .option("maxFilesPerTrigger", 1) 
                    .option("cloudFiles.format", "csv") 
                    .option("header", "true") 
                    .load(landing_zone + "/gym_logins_bz") 
                    .withColumn("load_time", F.current_timestamp())
                    .withColumn("source_file", F.input_file_name())
                )



    # Use append mode because bronze layer is expected to insert only from source

    stream_writer = df_stream.writeStream \
                                .format("delta") \
                                .option("checkpointLocation", checkpoint_base + "/gym_logins_bz") \
                                .outputMode("append") \
                                .queryName("gym_logins_bz_ingestion_stream")

    
    if once == True:
        return stream_writer.trigger(availableNow=True).toTable(f"{catalog}.{db_name}.gym_logins_bz")
    else:
        return stream_writer.trigger(processingTime=processing_time).toTable(f"{catalog}.{db_name}.gym_logins_bz")



def consume_kafka_multiplex(once = True, processing_time = "5 seconds"):
    schema = "key string, value string, topic string, partition bigint, offset bigint, timestamp bigint"
    df_date_lookup = spark.table(f"{catalog}.{db_name}.date_lookup").select("date", "week_part")

    df_stream = (spark.readStream
                    .format("cloudFiles")
                    .schema(schema)
                    .option("maxFilesPerTrigger", 5)
                    .option("cloudFiles.format", "json")
                    .load(landing_zone + "/kafka_multiplex_bz")                        
                    .withColumn("load_time", F.current_timestamp())       
                    .withColumn("source_file", F.input_file_name())
                    .join(F.broadcast(df_date_lookup), 
                            [F.to_date((F.col("timestamp")/1000).cast("timestamp")) == F.col("date")], 
                            "left")
                )

    # Use append mode because bronze layer is expected to insert only from source
    stream_writer = df_stream.writeStream \
                                .format("delta") \
                                .option("checkpointLocation", checkpoint_base + "/kafka_multiplex_bz") \
                                .outputMode("append") \
                                .queryName("kafka_multiplex_bz_ingestion_stream")

    

    if once == True:
        return stream_writer.trigger(availableNow=True).toTable(f"{catalog}.{db_name}.kafka_multiplex_bz")
    else:
        return stream_writer.trigger(processingTime=processing_time).toTable(f"{catalog}.{db_name}.kafka_multiplex_bz")
    



def consume_bronze(once=True, processing_time="5 seconds"):
    import time
    start = int(time.time())
    print(f"\nStarting bronze layer consumption ...")
    consume_user_registration(once, processing_time) 
    consume_gym_logins(once, processing_time) 
    consume_kafka_multiplex(once, processing_time)
    if once:
        for stream in spark.streams.active:
            stream.awaitTermination()
    print(f"Completed bronze layer consumtion {int(time.time()) - start} seconds")




def assert_count(catalog, db_name, table_name, expected_count, filter="true"):
    print(f"Validating record counts in {table_name}...", end='')
    actual_count = spark.read.table(f"{catalog}.{db_name}.{table_name}").where(filter).count()
    assert actual_count == expected_count, f"Expected {expected_count:,} records, found {actual_count:,} in {table_name} where {filter}" 
    print(f"Found {actual_count:,} / Expected {expected_count:,} records where {filter}: Success")        
    
def validate_bronze(catalog, db_name, sets):
    import time
    start = int(time.time())
    print(f"\nValidating bronz layer records...")
    assert_count(catalog, db_name,"registered_users_bz", 5 if sets == 1 else 10)
    assert_count(catalog, db_name,"gym_logins_bz", 8 if sets == 1 else 16)
    assert_count(catalog, db_name,"kafka_multiplex_bz", 7 if sets == 1 else 13, "topic='user_info'")
    assert_count(catalog, db_name,"kafka_multiplex_bz", 16 if sets == 1 else 32, "topic='workout'")
    assert_count(catalog, db_name,"kafka_multiplex_bz", sets * 253801, "topic='bpm'")
    print(f"Bronze layer validation completed in {int(time.time()) - start} seconds") 